<a href="https://colab.research.google.com/github/jrconstain/multilingual-sentiment-analysis/blob/main/Sentiment_Analysis_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de Sentimiento en Español con BERT

El siguiente código implementa una versión de BERT optimizada para el análisis de sentimiento multilenguaje (Español, Inglés, Frances, Italiano, Alemán y Holandes).

In [1]:
# Instalar librería Transformers
# Proporciona APIs y herramientas para descargar y trabajar con modelos pre-entrenados
!pip install transformers

# Importar librerías de trabajo
from transformers import pipeline
import pandas as pd # para bases de datos
import numpy as np # para cálculos matemáticos
import re # para detectar expresiones regulares en el texto

In [2]:
# Cargar datos de trabajo
resenas = pd.read_csv('resena_productos.csv')
tweets = pd.read_csv('tweets_politicos.csv') # crea dataframe 'tweets' con la info de tweets.xlsx

print(resenas.head()) # visualizar primeras filas de datos
print(tweets.head()) # visualizar primeras filas de datos

                                                text       valence
0  Me encantó este celular, la batería dura todo ...  muy positiva
1  Buen producto, cumple con lo prometido aunque ...      positiva
2  Es un artículo normal, nada especial pero tamp...        neutra
3  La calidad deja mucho que desear, no lo recome...      negativa
4    Horrible, se dañó en una semana. No lo compren.  muy negativa
                                                text       valence
0  El discurso de hoy fue inspirador y lleno de e...  muy positiva
1  Me gustó lo que dijo sobre la economía, parece...      positiva
2           Más de lo mismo, nada nuevo bajo el sol.        neutra
3  Promesas vacías como siempre, ya sabemos cómo ...      negativa
4  Es un desastre, cada palabra que dice es una m...  muy negativa


In [3]:
# Definir una función para pre-procesar el texto
def limpiar_texto(text):
    text = text.str.lower()  # Convertir a minúsculas
    text = text.str.replace(r'@\w+', '', regex=True)  # Eliminar nombres de usuario
    text = text.str.replace('\n', ' ', regex=False)  # Eliminar saltos de línea
    text = text.str.replace(r'http\S+', '', regex=True)  # Eliminar hipervínculos

    return text

# Aplicar la función en las columnas de texto de los df
tweets['text'] = limpiar_texto(tweets['text'])
resenas['text'] = limpiar_texto(resenas['text'])

# Visualizar las primeras entradas
print(tweets['text'].head())
print(resenas['text'].head())

0    el discurso de hoy fue inspirador y lleno de e...
1    me gustó lo que dijo sobre la economía, parece...
2             más de lo mismo, nada nuevo bajo el sol.
3    promesas vacías como siempre, ya sabemos cómo ...
4    es un desastre, cada palabra que dice es una m...
Name: text, dtype: object
0    me encantó este celular, la batería dura todo ...
1    buen producto, cumple con lo prometido aunque ...
2    es un artículo normal, nada especial pero tamp...
3    la calidad deja mucho que desear, no lo recome...
4      horrible, se dañó en una semana. no lo compren.
Name: text, dtype: object


In [4]:
# Importar clasificador desde https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment
classifier = pipeline('sentiment-analysis', model="nlptown/bert-base-multilingual-uncased-sentiment")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:
# Definir una función para evaluar el sentimiento
def get_sentiment(text):
    # Si el texto es NaN o no es texto, devuelve NaN
    if pd.isnull(text) or not isinstance(text, str):
        return np.nan

    result = classifier([text])[0]  # Guarda el resultado pasar el texto por el clasificador
    return int(result['label'].split()[0])  # Extrae el puntaje y lo convierte a número entero

In [6]:
# Aplicar la función a cada texto
resenas['sentiment'] = resenas['text'].apply(get_sentiment)
tweets['sentiment'] = tweets['text'].apply(get_sentiment)

# Primeras filas del resultado
print(resenas[['text','sentiment']])
print(tweets[['text','sentiment']])

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


                                                 text  sentiment
0   me encantó este celular, la batería dura todo ...          5
1   buen producto, cumple con lo prometido aunque ...          4
2   es un artículo normal, nada especial pero tamp...          3
3   la calidad deja mucho que desear, no lo recome...          2
4     horrible, se dañó en una semana. no lo compren.          1
5   funciona bien y tiene buen diseño, me gustó ba...          4
6   es decente, pero creo que hay mejores opciones...          3
7   estoy muy decepcionado, esperaba algo mejor po...          2
8   increíble, superó mis expectativas en todo sen...          5
9   es lo que se anuncia, no más ni menos. cumple ...          4
10  se siente barato y frágil, no lo volvería a co...          2
11  ¡el mejor que he probado! lo recomiendo sin du...          5
                                                 text  sentiment
0   el discurso de hoy fue inspirador y lleno de e...          5
1   me gustó lo que dijo 

In [10]:
# Mapeo de variable valence a su valor numérico equivalente
valence_map = {
    "muy negativa": 1,
    "negativa": 2,
    "neutra": 3,
    "positiva": 4,
    "muy positiva": 5
}

# Se define una función para evaluar el trabajo de BERT
def evaluar_sentimiento(row):
    valence = valence_map.get(row['valence'], 0)  # declara valence como el valor numérico de la variable valence utilizando el mapeo anterior
    sentiment = row['sentiment'] # declara sentiment como el valor de la variable sentiment

    # Evalua si valence y sentimiento son iguales (1), si son diferentes por un nivel (0.5), si son diferentes por más de un nivel (0)
    return 1 if valence == sentiment else 0.5 if abs(valence - sentiment) == 1 else 0

# Aplicar la función a los dataframes
resenas['eval'] = resenas.apply(evaluar_sentimiento, axis=1)
tweets['eval'] = tweets.apply(evaluar_sentimiento, axis=1)

print(resenas[['text', 'valence', 'sentiment', 'eval']])
print(tweets[['text', 'valence', 'sentiment', 'eval']])


                                                 text       valence  \
0   me encantó este celular, la batería dura todo ...  muy positiva   
1   buen producto, cumple con lo prometido aunque ...      positiva   
2   es un artículo normal, nada especial pero tamp...        neutra   
3   la calidad deja mucho que desear, no lo recome...      negativa   
4     horrible, se dañó en una semana. no lo compren.  muy negativa   
5   funciona bien y tiene buen diseño, me gustó ba...      positiva   
6   es decente, pero creo que hay mejores opciones...        neutra   
7   estoy muy decepcionado, esperaba algo mejor po...      negativa   
8   increíble, superó mis expectativas en todo sen...  muy positiva   
9   es lo que se anuncia, no más ni menos. cumple ...        neutra   
10  se siente barato y frágil, no lo volvería a co...      negativa   
11  ¡el mejor que he probado! lo recomiendo sin du...  muy positiva   

    sentiment  eval  
0           5   1.0  
1           4   1.0  
2         

In [13]:
print(f"Punta para las reseñas: {resenas['eval'].sum()}")
print(f"Punta para los tweets: {tweets['eval'].sum()}")

Punta para las reseñas: 11.5
Punta para los tweets: 8.0
